# REDES NEURAIS ARTIFICIAIS

# PARTE 1: Classificação Binária - Diagnóstico de Câncer de Mama

Base de dados: Breast Cancer Wisconsin (Diagnostic) -
https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

In [1]:
#bilbiotecas keras e tensorflow
%pip install scikeras==0.13.0

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.0/1.5 MB 487.6 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.5 MB 327.7 kB/s eta 0:00:05
   - -------------------------------------- 0.1/1.5 MB 409.6 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.5 MB 544.7 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.5 MB 544.7 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.5 MB 546.6 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.5 MB 546.6 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.5 MB 546.6 kB/s eta 0:00:03
   ------ --------------------------------- 0.2/1.5 MB 550.0 kB/s eta 0:00:03
   ------ --------------------------------- 0.2/1.5 MB 558.1 kB/s eta 0:00:03
   ---

In [3]:
%pip install tensorflow==2.16.1 scikit-learn==1.5.0

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.9 MB 660.6 kB/s eta 0:00:17
   ---------------------------------------- 0.1/10.9 MB 656.4 kB/s eta 0:00:17
   ---------------------------------------- 0.1/10.9 MB 655.4 kB/s eta 0:00:17
   ---------------------------------------- 0.1/10.9 MB 656.4 kB/s eta 0:00:17
   ---------------------------------------- 0.1/10.9 MB 599.1 kB/s eta 0:00:19
    --------------------------------------- 0.1/10.9 MB 532.5 kB/s eta 0:00:21
    --------------------------------------- 0.2/10.9 MB 482.7 kB/s eta 0:00:23
    --------------------------------------- 0.2/10.9 MB 455.1 kB/s eta 0:00:24
    --------------------------------------- 0.2/10.9 MB 479.2 kB/s eta 0:00:23
    --------------------------------------- 0.2/10.9 MB 479.2 kB/s eta 0:00:23
 

In [ ]:
# após instalar as bibliotecas, é necessário reiniciar o notebook
# Para isso, rode esta célula sozinha antes de prosseguir.
exit()

: 

In [1]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras
from sklearn.model_selection import train_test_split

In [2]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.3.3', '2.16.1', '1.5.0', '0.13.0')

### Carregar os dados

1) Carregue a base de dados, faça a divisição de treino e teste (para isso, utilize a função train_test_split do sklearn), como o tamanho da base de teste de 0.25.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
dados = pd.read_csv("dados_breast.csv")
rotulos = pd.read_csv("rotulos_breast.csv")
(dados, rotulos), _ = fashion_mnist.load_data()

X = dados / 255.0
y = rotulos

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

In [24]:
X_treinamento.shape, y_treinamento.shape

((45000, 28, 28), (45000,))

In [25]:
X_teste.shape, y_teste.shape

((15000, 28, 28), (15000,))

### Estrutura da Rede Neural Artificial e Teste

In [6]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import backend as k
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

2) Crie a RNA com as seguintes configurações:

a) uma camada de entrada de com 30 neurônios;

b) uma camada oculta densa com 16 neurônios. Porque a camada oculta tem essa quantidade de neurônios?

c) adicione na camada oculta a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer (https://www.tensorflow.org/api_docs/python/tf/keras/initializers);

d) adicione a camada de saíde com a função de ativação sigmoid. Porque foi utilizada esta função para a saída da RNA?

3) Porque utilizamos a classe chamada Sequential para a RNA?

In [ ]:
'''
b) Geralmente escolhemos um número que seja suficiente para a rede aprender padrões complexos, mas não tão grande que torne o treinamento lento ou cause overfitting
d) A função Sigmoid é usada porque ela transforma qualquer valor que venha dos neurônios anteriores em um número entre 0 e 1
3) Usamos a classe Sequential porque ela é a maneira mais simples e direta de construir modelos no keras, onde as camadas são empilhadas em linha reta, uma após a outra
'''
rede_neural = Sequential()

# Camada de entrada — 30 neurônios
rede_neural.add(Dense(
    units=30,
    activation='relu',
    kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05),
    input_shape=([28, 28])
))

# Camada oculta — 16 neurônios
rede_neural.add(Dense(
    units=16,
    activation='relu',
    kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05)
))

# Camada de saída — 1 neurônio (classificação binária)
rede_neural.add(Dense(
    units=1,
    activation='sigmoid'
))

4) A partir da RNA gerada, explique o que são os valores apresentados na tabela da rede_neural.summary()

In [27]:
'''
4)
Layer: Mostra o nome e o tipo da camada
Output Shape: Mostra a dimensão dos dados que saem dessa camada
Param #: É a quantidade de ajustes ou botões internos que a rede vai aprender durante o treinamento
'''
rede_neural.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 28, 30)         │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 28, 16)         │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 28, 1)          │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,383 (5.40 KB)

 Trainable params: 1,383 (5.40 KB)

 Non-trainable params: 0 (0.00 B)

5) Adicione um otimizador Adam (https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) e especifique a classe loss binário - binary crossentropy (https://www.tensorflow.org/api_docs/python/tf/keras/losses) e a classe metrics para utilizar a métrica de avaliação de acurácia binária (https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

6) Para que servem os otimizadores? Como o otimizador Adam funciona?

7) Depois de estruturados os parâmtros da RNA, utilize a função .fit para fazer o treinamento da rede. Como foi utilizado o otimizador Adam, e ele é baseado na descida do gradiente estocástica, é possível definir a quantidade de registros que serão enviados para a RNA, isto é, em cada batch serão utilizados 10 registros. Quantos batches serão utilizados ao total?

8) Por fim, defina o número de épocas em que ocorre o treinamento igual a 100.

In [28]:
'''
6) os otimizadores são o motor de aprendizado da rede neural
O otimizador usa o cálculo do rgadiente (a inclinação do erro) para determinar
a direção e a intensidade com que os pesos internos da rede devem ser ajustados 
para que o erro diminua
ou seja, ele garante que a rede se mova em direção ao ponto onde o erro é o menor possível
7)o tamanho do lote é 10 e os dados de treinamento são 45.000 registros, como vamos enviar os
dados em pacotes de 10 registros por lote (batch), a rede utilizará 4.500 lotes por época
a cada vez que a rede completa uma época, ela executa 4.500 passos de ajuste para aprender
com todos os dados
o treinamento total será de 4.500 lotes x 100 épocas = 450.000 lotes
'''

rede_neural.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=[BinaryAccuracy()]
)


historia = rede_neural.fit(
    X_treinamento, 
    y_treinamento, 
    
    batch_size=10, 
    epochs=100
)

Epoch 1/100


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(10,), output.shape=(10, 28, 1)

9) Crie uma variável chamada previsoes para realizar a previsão dos dados de teste (X_teste) O resultado da rede deve ser um valor entre 0 e 1. Porque isso acontece?

In [15]:
'''
9)isso acontece por causa da ativação utilizada na camada de saída da rede neural, a função sigmoid
'''
previsoes = rede_neural.predict(X_teste)
previsoes

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

10) Converta o resultado que está em probabilidade para valores binários com um limiar (treshold) de 0.5.


In [11]:
previsoes = (previsoes >= 0.5).astype(int)

In [12]:
previsoes

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

9) Explique o resultado da RNA.

In [16]:
'''
9) O resultado é um loss de -
'''
rede_neural.evaluate(X_teste, y_teste)

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.1031 - loss: -960395712.0000


[-960395712.0, 0.10306666791439056]

## Camadas e Otimização da RNA

10) Adicione mais uma camada para oculta densa com 16 neurônio com a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer. Qual o total de parâmetros da RNA agora?

In [ ]:
rede_neural.summary()

11) Abaixo são adicionados os parâmetros do otimizador, que são a taxa de aprendizado e o clipvalue. O que eles fazem?

In [ ]:
otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

12) Teste novamente a RNA. Aumentar a quantidade de camadas melhorou ou piorou os resultados? Explique o que aconteceu com a RNA e porque.

In [ ]:
rede_neural.fit(X_treinamento, y_treinamento, batch_size = 10, epochs = 100)

In [ ]:
previsoes = rede_neural.predict(X_teste)

In [ ]:
rede_neural.evaluate(X_teste, y_teste)

## K-Fold Cross Validation

13) Assista o vídeo https://youtu.be/RczbeFs_WbQ?si=f1Yu4LZaGDfZZpbZ para compreender melhor sobre como funciona o k-fold cross validation. Explique como esta rede foi configurada e como é possível chegar no resultado dela. O que é necessário fazer?


14) Calcule também o Desvio Padrão dos resultados para avaliar o modelo. O que é possível concluir com esse resultado?



In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k

In [ ]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [ ]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

In [ ]:
resultados

## Overfitting e Dropout

14) Como vimos, o modelo anterior sofreu overfitting. Para reduzir esse problema, aplicamos a técnica de regularização conhecida como dropout (artigo oficial: https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf e explicação em português: https://www.deeplearningbook.com.br/como-funciona-o-dropout/ ). Aplique o dropout de 20% na primeira e segunda camada oculta. O que acontece com os resultados? E o Desvio Padrão?

In [ ]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [ ]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

In [ ]:
resultados

## Tuning dos Hiperparâmetros (técnicas de ajuste)

Hyperparameter Tuning (Ajuste de Hiperparâmetros) é o processo geral de encontrar os melhores valores para os hiperparâmetros de um modelo de machine learning.
Exemplos de hiperparâmetros:

- Taxa de aprendizado (learning rate)

- Número de árvores em um RandomForest

- Número de camadas ou neurônios em uma rede neural

- Tipo de kernel em um SVM

O tuning pode ser feito por diferentes técnicas, como:

- Grid Search

- Random Search

- Bayesian Optimization, etc.

15) Descreva como a RNA foi configurada para fazer o processo de tuning.

In [ ]:
def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])
  return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede)

In [ ]:
parametros = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'model__optimizer': ['adam', 'sgd'],
    'model__loss': ['binary_crossentropy', 'hinge'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu', 'tanh'],
    'model__neurons': [16, 8]
}

In [ ]:
#caso tiver problema com a configuração anterior, testar essa
parametros = {
    'batch_size': [10, 30],
    'epochs': [50],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu'],
    'model__neurons': [16]
}

In [ ]:
parametros

In [ ]:
grid_search = GridSearchCV(estimator = rede_neural, param_grid = parametros,
                          scoring = 'accuracy', cv = 5)

In [ ]:
grid_search = grid_search.fit(X, y)

In [ ]:
print(grid_search)

In [ ]:
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

In [ ]:
melhor_precisao = grid_search.best_score_
print(melhor_precisao)

16) É possível melhorar ainda mais a curácia da RNA? Como?

#Para salvar o seu modelo de RNA

In [ ]:
classificador.save('classificador_breast.keras')

In [ ]:
classificador_novo = tf.keras.models.load_model('/content/classificador_breast.keras')

In [ ]:
classificador_novo.summary()